In [21]:
import pandas as pd
import numpy as np
import requests
import re

import openrouteservice
import json
import geopy

In [31]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Route Revealer")
location = geolocator.geocode("869 Burlington, Fillmore, CA, 93015") #ex

print((location.longitude, location.latitude))


(-118.92084123989956, 34.39196708819081)


In [2]:
body = {"locations":[[9.70093,48.477473],[9.207916,49.153868],[37.573242,55.801281]],"metrics":["distance","duration"],"units":"mi"}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf6248671670378c684678936d09cf57ef7b50',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

print(call.status_code, call.reason)
print(call.text)

200 OK
{"durations":[[0.0,5755.66,88843.65],[5627.36,0.0,86959.81],[88247.25,86226.88,0.0]],"distances":[[0.0,87.53,1512.56],[86.85,0.0,1482.41],[1479.78,1449.99,0.0]],"destinations":[{"location":[9.700817,48.476406],"snapped_distance":118.92},{"location":[9.207773,49.153882],"snapped_distance":10.54},{"location":[37.572963,55.801279],"snapped_distance":17.45}],"sources":[{"location":[9.700817,48.476406],"snapped_distance":118.92},{"location":[9.207773,49.153882],"snapped_distance":10.54},{"location":[37.572963,55.801279],"snapped_distance":17.45}],"metadata":{"attribution":"openrouteservice.org | OpenStreetMap contributors","service":"matrix","timestamp":1594154528968,"query":{"locations":[[9.70093,48.477473],[9.207916,49.153868],[37.573242,55.801281]],"profile":"driving-car","responseType":"json","metricsStrings":["DISTANCE","DURATION"],"metrics":["distance","duration"],"units":"mi"},"engine":{"version":"6.1.1","build_date":"2020-06-18T15:14:48Z","graph_date":"2020-07-01T08:16:27Z"}}

In [3]:
output = json.loads(call.text)
output

{'durations': [[0.0, 5755.66, 88843.65],
  [5627.36, 0.0, 86959.81],
  [88247.25, 86226.88, 0.0]],
 'distances': [[0.0, 87.53, 1512.56],
  [86.85, 0.0, 1482.41],
  [1479.78, 1449.99, 0.0]],
 'destinations': [{'location': [9.700817, 48.476406],
   'snapped_distance': 118.92},
  {'location': [9.207773, 49.153882], 'snapped_distance': 10.54},
  {'location': [37.572963, 55.801279], 'snapped_distance': 17.45}],
 'sources': [{'location': [9.700817, 48.476406], 'snapped_distance': 118.92},
  {'location': [9.207773, 49.153882], 'snapped_distance': 10.54},
  {'location': [37.572963, 55.801279], 'snapped_distance': 17.45}],
 'metadata': {'attribution': 'openrouteservice.org | OpenStreetMap contributors',
  'service': 'matrix',
  'timestamp': 1594154528968,
  'query': {'locations': [[9.70093, 48.477473],
    [9.207916, 49.153868],
    [37.573242, 55.801281]],
   'profile': 'driving-car',
   'responseType': 'json',
   'metricsStrings': ['DISTANCE', 'DURATION'],
   'metrics': ['distance', 'duration

In [16]:
output['distances'][1]

[86.85, 0.0, 1482.41]

In [20]:
[round(i/60,2) for i in output['durations'][1]]

[93.79, 0.0, 1449.33]

## Example

In [116]:
df = pd.read_csv('/Users/ryan/Desktop/test_route.txt')
df

,Departure Street,D City,D State,D Zip,Equipment Street,E City,E State,E Zip,WorkSite Street,W City,W State,W Zip
0,869 Burlington Street,Fillmore,CA,93015,389 N Brent Street,Ventura,CA,93003,763 Burlington St,Fillmore,CA,93015
1,7140 Winding Way,Fair Oaks,CA,95628,557 Mockingbird Lane,Fillmore,CA,93015,6675 Morse Ave,North Hollywood,CA,91606


In [117]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df #stripped of whitespaces!

,Departure Street,D City,D State,D Zip,Equipment Street,E City,E State,E Zip,WorkSite Street,W City,W State,W Zip
0,869 Burlington Street,Fillmore,CA,93015,389 N Brent Street,Ventura,CA,93003,763 Burlington St,Fillmore,CA,93015
1,7140 Winding Way,Fair Oaks,CA,95628,557 Mockingbird Lane,Fillmore,CA,93015,6675 Morse Ave,North Hollywood,CA,91606


In [118]:
departure = df[['Departure Street','D State','D Zip']]

addresses_d = []
for i in range(departure.shape[0]):
    addresses_d.append(','.join(map(str, departure.iloc[i].values)))
addresses_d 

lat_long_d = []
for i in range(len(addresses_d)):
    location = geolocator.geocode(addresses_d[i])
    lat_long_d.append([location.longitude, location.latitude])
    print((location.longitude, location.latitude))

(-118.92084123989956, 34.39196708819081)
(-121.29714466666667, 38.64904383333334)


In [120]:
equipment = df[['Equipment Street','E State','E Zip']]
equipment

addresses_e = []
for i in range(equipment.shape[0]):
    addresses_e.append(','.join(map(str, equipment.iloc[i].values)))
addresses_e

lat_long_e = []
for i in range(len(addresses_e)):
    location = geolocator.geocode(addresses_e[i])
    lat_long_e.append([location.longitude, location.latitude])
    print((location.longitude, location.latitude))


(-119.25844558108108, 34.278454675675675)
(-118.930412, 34.402202)


In [121]:
worksite = df[['WorkSite Street','W State','W Zip']]
worksite

addresses_w = []
for i in range(worksite.shape[0]):
    addresses_w.append(','.join(map(str, worksite.iloc[i].values)))
addresses_w

lat_long_w = []
for i in range(len(addresses_w)):
    location = geolocator.geocode(addresses_w[i])
    lat_long_w.append([location.longitude, location.latitude])
    print((location.longitude, location.latitude))

(-118.91837712168271, 34.39226241832958)
(-118.417158, 34.1925)


In [128]:
combined_df_coords = pd.concat([pd.DataFrame(lat_long_d), pd.DataFrame(lat_long_e), pd.DataFrame(lat_long_w)],axis=1)
combined_df_coords

,0,1,0,1,0,1
0,-118.920841,34.391967,-119.258446,34.278455,-118.918377,34.392262
1,-121.297145,38.649044,-118.930412,34.402202,-118.417158,34.192500


In [ ]:
# need to turn below cell into a loop

In [141]:
input_api = np.array_split(combined_df_coords.iloc[0].values,combined_df_coords.shape[1]/2)
input_api = [i.tolist() for i in input_api]
input_api

[[-118.92084123989956, 34.39196708819081],
 [-119.25844558108108, 34.278454675675675],
 [-118.91837712168271, 34.39226241832958]]

In [142]:
body = {"locations":input_api,"metrics":["distance","duration"],"units":"mi"}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf6248671670378c684678936d09cf57ef7b50',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

print(call.status_code, call.reason)
print(call.text)

200 OK
{"durations":[[0.0,1793.22,55.14],[1798.2,0.0,1835.56],[55.14,1824.12,0.0]],"distances":[[0.0,22.94,0.14],[23.01,0.0,23.41],[0.14,23.33,0.0]],"destinations":[{"location":[-118.920828,34.391883],"snapped_distance":9.47},{"location":[-119.258368,34.278466],"snapped_distance":7.21},{"location":[-118.918351,34.392179],"snapped_distance":9.6}],"sources":[{"location":[-118.920828,34.391883],"snapped_distance":9.47},{"location":[-119.258368,34.278466],"snapped_distance":7.21},{"location":[-118.918351,34.392179],"snapped_distance":9.6}],"metadata":{"attribution":"openrouteservice.org | OpenStreetMap contributors","service":"matrix","timestamp":1594166573655,"query":{"locations":[[-118.92084123989956,34.39196708819081],[-119.25844558108108,34.278454675675675],[-118.91837712168271,34.39226241832958]],"profile":"driving-car","responseType":"json","metricsStrings":["DISTANCE","DURATION"],"metrics":["distance","duration"],"units":"mi"},"engine":{"version":"6.1.1","build_date":"2020-06-18T15:

In [144]:
output = json.loads(call.text)
print('Miles:')
print(output['distances'][1])
print('Time:')
print([round(i/60,2) for i in output['durations'][1]])

Miles:
[23.01, 0.0, 23.41]
Time:
[29.97, 0.0, 30.59]
